In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [98.9 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hi

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-07 14:07:22--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.50MB/s    in 0.2s    

2022-11-07 14:07:23 (4.50 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NFL-Big-Data-Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Store environment variable
from getpass import getpass
password = getpass('Enter database password')

# Configure settings for Cloud SQL
mode = "append"
jdbc_url="jdbc:postgresql://34.72.136.99:5432/big-data-bowl"
config = {"user":"robert", 
          "password": password, 
          "driver":"org.postgresql.Driver"}


Enter database password··········


In [5]:
# Read in data from GCS Buckets
from pyspark import SparkFiles
games_data_df = spark.read.jdbc(url=jdbc_url,table='games',properties=config)

# Show DataFrame
games_data_df.show()

+----------+------+----+----------+---------------+------------+---------------+
|    gameId|season|week|  gameDate|gameTimeEastern|homeTeamAbbr|visitorTeamAbbr|
+----------+------+----+----------+---------------+------------+---------------+
|2021090900|  2021|   1|09/09/2021|       20:20:00|          TB|            DAL|
|2021091200|  2021|   1|09/12/2021|       13:00:00|         ATL|            PHI|
|2021091201|  2021|   1|09/12/2021|       13:00:00|         BUF|            PIT|
|2021091202|  2021|   1|09/12/2021|       13:00:00|         CAR|            NYJ|
|2021091203|  2021|   1|09/12/2021|       13:00:00|         CIN|            MIN|
|2021091204|  2021|   1|09/12/2021|       13:00:00|         DET|             SF|
|2021091205|  2021|   1|09/12/2021|       13:00:00|         HOU|            JAX|
|2021091206|  2021|   1|09/12/2021|       13:00:00|         IND|            SEA|
|2021091207|  2021|   1|09/12/2021|       13:00:00|         TEN|            ARI|
|2021091208|  2021|   1|09/1

In [6]:
games_data_df.dtypes

[('gameId', 'int'),
 ('season', 'int'),
 ('week', 'int'),
 ('gameDate', 'string'),
 ('gameTimeEastern', 'string'),
 ('homeTeamAbbr', 'string'),
 ('visitorTeamAbbr', 'string')]

In [12]:
gb_games_df = games_data_df.filter((games_data_df.homeTeamAbbr=="GB") | (games_data_df.visitorTeamAbbr=="GB"))
gb_games_df.show()

+----------+------+----+----------+---------------+------------+---------------+
|    gameId|season|week|  gameDate|gameTimeEastern|homeTeamAbbr|visitorTeamAbbr|
+----------+------+----+----------+---------------+------------+---------------+
|2021091211|  2021|   1|09/12/2021|       16:25:00|          NO|             GB|
|2021092000|  2021|   2|09/20/2021|       20:15:00|          GB|            DET|
|2021092613|  2021|   3|09/26/2021|       20:20:00|          SF|             GB|
|2021100312|  2021|   4|10/03/2021|       16:25:00|          GB|            PIT|
|2021101002|  2021|   5|10/10/2021|       13:00:00|         CIN|             GB|
|2021101703|  2021|   6|10/17/2021|       13:00:00|         CHI|             GB|
|2021102401|  2021|   7|10/24/2021|       13:00:00|          GB|            WAS|
|2021102800|  2021|   8|10/28/2021|       20:20:00|         ARI|             GB|
+----------+------+----+----------+---------------+------------+---------------+

